In [1]:
import numpy as np
# import matplot for visualization
import matplotlib.pyplot as plt
#to see the plots inside of the notebook
%matplotlib inline
# import tensorflow and required layers
# note that tensorflow.contrib.layers was previously migrated from TF Slim.
import tensorflow as tf
from tensorflow.contrib.layers import conv2d, fully_connected, max_pool2d, repeat
from scipy import misc
slim = tf.contrib.slim


# create a session
session = tf.Session()

In [2]:
# We begin with some preprocessing code to load the ImageNet class names.
# This code is from https://github.com/tensorflow/models/blob/master/slim/datasets/imagenet.py

filename = "data/IMAGENET/predictions_to_labels.txt"
synset_list = [s.strip() for s in open(filename).readlines()]
num_synsets_in_ilsvrc = len(synset_list)
assert num_synsets_in_ilsvrc == 1000

filename = "data/IMAGENET/labels_to_human.txt"
synset_to_human_list = open(filename).readlines()
num_synsets_in_all_imagenet = len(synset_to_human_list)
assert num_synsets_in_all_imagenet == 21842

synset_to_human = {}
for s in synset_to_human_list:
    parts = s.strip().split('\t')
    assert len(parts) == 2
    synset = parts[0]
    human = parts[1]
    synset_to_human[synset] = human

label_index = 0
labels_to_names = dict()

for synset in synset_list:
    name = synset_to_human[synset]
    labels_to_names[label_index] = name
    label_index += 1

In [3]:

# Network definition copied from https://github.com/tensorflow/models/blob/master/slim/nets/vgg.py
def vgg_16(inputs, num_classes=1000, scope='vgg_16'):
    
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        
        with slim.arg_scope([conv2d, fully_connected, max_pool2d]):
            net = repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = max_pool2d(net, [2, 2], scope='pool1')
            net = repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = max_pool2d(net, [2, 2], scope='pool2')
            net = repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
            net = max_pool2d(net, [2, 2], scope='pool3')
            net = repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
            net = max_pool2d(net, [2, 2], scope='pool4')
            net = repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
            #print(dir(net))
            #net = max_pool2d(net, [2, 2], scope='pool5')
            # Use conv2d instead of fully_connected layers.
            #net = conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
            #net = conv2d(net, 4096, [1, 1], scope='fc7')
            #net = conv2d(net, num_classes, [1, 1], activation_fn=None, normalizer_fn=None, scope='fc8')
            
            return net

In [4]:
# Create the placeholder for the inputs. Note that the input shape for vgg is 224 x 224 x 3 (in classification mode)
inputs = tf.placeholder(dtype=tf.float32,shape=(1,224,224,3))


# Build the network (It requires inputs and number of output classes) 
vgg = vgg_16(inputs)


# Transform logits to probabilities
#probs = tf.nn.softmax(vgg)

In [5]:
# Load 3 images
imgsDir = 'images/'
imgsType='.jpg'
imgsNames = ['fly','car','jet']
imgs = [misc.imresize(misc.imread(imgsDir+imgName+imgsType),size=[224,224]).reshape([1,224,224,3]) for imgName in imgsNames ]
# imgFly = misc.imread('images/fly.jpg')
# imgFly = misc.imresize(imgFly,size=[224,224])
# imgCar = misc.imread('images/car.jpg')
# imgCar = misc.imresize(imgCar,size=[224,224])
# imgJet = misc.imread('images/jet.jpg')
# imgJet = misc.imresize(imgJet,size=[224,224])

#print(img.shape)    # (32, 32, 3)

#img_tf = tf.Variable(img)
#print(img_tf.get_shape().as_list())  # [32, 32, 3]

# plt.imshow(imgFly)
# plt.show()

# plt.imshow(imgCar)
# plt.show()

# plt.imshow(imgJet)
# plt.show()


In [6]:
# Create the initialization function for the variables of the network
init_op = tf.global_variables_initializer()
session.run(init_op)

# Define an operator to load model variables from a checkpoint using Slim.
# The checkpoint can be found at https://github.com/tensorflow/models/tree/master/slim

# TODO restore the saved checkpoints ./checkpoints/model.ckpt

# Assuming than 'conv1/weights' should be restored from 'vgg16/conv1/weights'

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
#print_tensors_in_checkpoint_file(file_name='checkpoints/vgg_16.ckpt', tensor_name='', all_tensors='')

#tvars =tf.trainable_variables()
#print(tvars)

def name_in_checkpoint(var):
  if var.name.startswith('vgg'):
    return var.name.split(':')[0]
  return 'vgg_16/' + var.name.split(':')[0]


variables_to_restore = slim.get_model_variables()
variables_to_restore = {name_in_checkpoint(var):var for var in variables_to_restore}

cleaned = dict()

for var in slim.get_model_variables():
 
  if var.op.name.startswith('fully_connected') or var.op.name.startswith('Conv') :
    continue
  cleaned[name_in_checkpoint(var)]=var

#print(cleaned)

restorer = tf.train.Saver(cleaned)

with tf.Session() as sess:
  # Restore variables from disk.
  restorer.restore(session, "checkpoints/vgg_16.ckpt")






INFO:tensorflow:Restoring parameters from checkpoints/vgg_16.ckpt


In [13]:
# Start a tensorflow session and run the aforementioned operators
#imgJetReshaped = imgJet.reshape([1,224,224,3])

# #Predict the classes for the given images
#for img in imgs:
img=imgs[0]
probsOut = session.run([vgg],feed_dict={inputs:img})
print(probsOut[0][0,:,:,0].shape)
    #for idx in ((probsOut[0][0,0,0,:]).argsort()[-5:][::-1]):
    #    print(labels_to_names[idx]+': '+str(probsOut[0][0,0,0,:][idx]))
    #plt.imshow(img[0,:,:,:])
    #plt.show()
    




(14, 14)


In [ ]:
import numpy as np
# import matplot for visualization
import matplotlib.pyplot as plt
#to see the plots inside of the notebook
%matplotlib inline
# import tensorflow and required layers
# note that tensorflow.contrib.layers was previously migrated from TF Slim.
import tensorflow as tf
from tensorflow.contrib.layers import conv2d, fully_connected, max_pool2d, repeat
from scipy import misc
slim = tf.contrib.slim


# create a session
session = tf.Session()


# Load 3 images
imgsDir = 'images/'
imgsType='.jpg'
imgsNames = ['fly','car','jet']
imgs = [misc.imresize(misc.imread(imgsDir+imgName+imgsType),size=[224,224]).reshape([1,224,224,3]) for imgName in imgsNames ]
# imgFly = misc.imread('images/fly.jpg')
# imgFly = misc.imresize(imgFly,size=[224,224])
# imgCar = misc.imread('images/car.jpg')
# imgCar = misc.imresize(imgCar,size=[224,224])
# imgJet = misc.imread('images/jet.jpg')
# imgJet = misc.imresize(imgJet,size=[224,224])

#print(img.shape)    # (32, 32, 3)

#img_tf = tf.Variable(img)
#print(img_tf.get_shape().as_list())  # [32, 32, 3]

# plt.imshow(imgFly)
# plt.show()

# plt.imshow(imgCar)
# plt.show()

# plt.imshow(imgJet)
# plt.show()


init_op = tf.global_variables_initializer()
session.run(init_op)
inputs = tf.placeholder(dtype=tf.float32,shape=(1,224,224,3))
with tf.variable_scope('vgg_16', inputs, [1000]) as sc:
        with slim.arg_scope([conv2d, fully_connected, max_pool2d]):
            net = repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = max_pool2d(net, [2, 2], scope='pool1')
            net = repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = max_pool2d(net, [2, 2], scope='pool2')
            net = repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
            net = max_pool2d(net, [2, 2], scope='pool3')
            net = repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
            net = max_pool2d(net, [2, 2], scope='pool4')
            net = repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
            dir(net)
            #net = max_pool2d(net, [2, 2], scope='pool5')
            # Use conv2d instead of fully_connected layers.
            #net = conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
            #net = conv2d(net, 4096, [1, 1], scope='fc7')
            #net = conv2d(net, num_classes, [1, 1], activation_fn=None, normalizer_fn=None, scope='fc8')
            for img in imgs:
                probsOut = session.run([net],feed_dict={inputs:img})